<h1 div class='alert alert-success'><center> Tunning Hyperparameters XGB
 </center></h1>

![](https://storage.googleapis.com/kaggle-competitions/kaggle/26480/logos/header.png?t=2021-04-09-00-57-05)

# <div class="alert alert-success">  0. IMPORTAÇÕES </div>

In [ ]:
!pip install --quiet optuna
!pip install --q GPUtil
!pip install --q BorutaShap

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 0.1. Bibliotecas

In [1]:
import warnings
import os
import gc
import random
import glob
import optuna

In [2]:
import pandas               as pd
import numpy                as np
import matplotlib.pyplot    as plt 
import seaborn              as sns
import joblib               as jb

In [3]:
import torch
import torch.nn             as nn

In [4]:
import xgboost              as xgb

In [5]:
from sklearn.model_selection       import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing         import QuantileTransformer, StandardScaler, KBinsDiscretizer
from sklearn.preprocessing         import RobustScaler, MinMaxScaler, MaxAbsScaler, LabelEncoder
from sklearn                       import metrics
from sklearn.feature_selection     import SelectKBest, SelectPercentile, f_classif

In [6]:
from optuna.samplers               import TPESampler
from optuna.visualization          import plot_edf
from optuna.visualization          import plot_optimization_history
from optuna.visualization          import plot_parallel_coordinate
from optuna.visualization          import plot_param_importances
from optuna.visualization          import plot_slice
from optuna.visualization          import plot_intermediate_values
from optuna.visualization          import plot_contour
from optuna.pruners                import MedianPruner

In [7]:
from GPUtil                        import showUtilization as gpu_usage
from numba                         import cuda
from sklearn.ensemble              import IsolationForest
from psutil                        import virtual_memory
from datetime                      import datetime
from psutil                        import virtual_memory
from BorutaShap                    import BorutaShap
from sklearn.utils.class_weight    import compute_sample_weight

## 0.2. Funções

In [8]:
def jupyter_setting():
    
    %matplotlib inline
     
    pd.options.display.max_columns = None
    
    optuna.logging.set_verbosity(optuna.logging.WARNING)
      
    warnings.filterwarnings(action='ignore')
    warnings.simplefilter('ignore')
    warnings.filterwarnings('ignore')
    warnings.filterwarnings('ignore', category=DeprecationWarning)
    warnings.filterwarnings('ignore', category=FutureWarning)
    warnings.filterwarnings('ignore', category=RuntimeWarning)
    warnings.filterwarnings('ignore', category=UserWarning)
    #pd.set_option('display.max_rows', 150)
    pd.set_option('display.max_columns', 500)
    pd.set_option('display.max_colwidth', None)

    icecream = ["#00008b", "#960018","#008b00", "#00468b", "#8b4500", "#582c00"]
    #sns.palplot(sns.color_palette(icecream))
    
    return icecream

icecream = jupyter_setting()

In [9]:
def reduce_memory_usage(df, verbose=True):
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [10]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
    plt.rcParams['font.size'] = 12
    plt.title('Precision Recall vs threshold')
    plt.xlabel('Threshold')
    plt.legend(loc="lower left")
    
    plt.grid(True)

In [11]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls[:-1], precisions[:-1], "b-", label="Precision")
    
    plt.rcParams['font.size'] = 12
    plt.title('Precision vs recall')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # plt.legend(loc="lower left")
    
    plt.grid(True)

In [12]:
def plot_roc_curve(fpr, tpr, label=None):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr, "r-", label=label)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="--", c=".3")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.rcParams['font.size'] = 12
    plt.title('XGBR ROC curve for TPS 09')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.legend(loc="lower right")
    plt.grid(True)

In [13]:
def graf_corr(df):
    
    df = df.corr().round(5)

    # Máscara para ocultar a parte superior direita do gráfico, pois é uma duplicata
    mask = np.zeros_like(df)
    mask[np.triu_indices_from(mask)] = True

    # Making a plot
    plt.figure(figsize=(16,16))
    ax = sns.heatmap(df, annot=True, mask=mask, cmap="RdBu", annot_kws={"weight": "bold", "fontsize":13})

    ax.set_title("Mapa de calor de correlação das variável", fontsize=17)

    plt.setp(ax.get_xticklabels(), 
             rotation      = 90, 
             ha            = "right",
             rotation_mode = "anchor", 
             weight        = "normal")

    plt.setp(ax.get_yticklabels(), 
             weight        = "normal",
             rotation_mode = "anchor", 
             rotation      = 0, 
             ha            = "right");

In [14]:
def correlation(dataset, threshold):

    col_corr    = set()  # Conjunto de todos os nomes de colunas correlacionadas
    corr_matrix = dataset.corr()
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if abs(corr_matrix.iloc[i, j]) >= threshold: # estamos interessados no valor coeficiente absoluto
                colname = corr_matrix.columns[i]        # obtendo o nome da coluna
                col_corr.add(colname)
    
    return col_corr

In [15]:
def free_gpu_cache():
    
    # https://www.kaggle.com/getting-started/140636
    #print("Initial GPU Usage")
    #gpu_usage()                             

    #cuda.select_device(0)
    #cuda.close()
    #cuda.select_device(0)   
    
    gc.collect()
    torch.cuda.empty_cache()

## 0.3. GPU

### 0.3.1. Informações

In [16]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)

if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Dec 26 09:34:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.06       Driver Version: 510.06       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A    0C    P8    N/A /  N/A |     78MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 0.3.2. Memória

In [17]:
ram_gb = virtual_memory().total / 1e9

print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 17.0 gigabytes of available RAM

Not using a high-RAM runtime


## 0.4. Carregar Dados

In [1]:
def mkdir():
    
    !mkdir img
    !mkdir Data
    !mkdir Data\pkl
    !mkdir Data\submission
    !mkdir Data\submission\tunning

    !mkdir model
    !mkdir model\mdl
    !mkdir model\preds
    !mkdir model\optuna

    !mkdir model\preds\test
    !mkdir model\preds\test\n1
    !mkdir model\preds\test\n2
    !mkdir model\preds\test\n3

    !mkdir model\preds\train
    !mkdir model\preds\train\n1
    !mkdir model\preds\train\n2
    !mkdir model\preds\train\n3
    !mkdir model\preds\param
    
mkdir 

In [19]:
path   = '' #'/content/drive/MyDrive/kaggle/Tabular Playground Series/2021/12 - Dezembro/'
target = 'Cover_Type'

In [20]:
df3_train     = jb.load(path+ 'Data/pkl/df2_nb_02_train.pkl.z')
df3_test      = jb.load(path+ 'Data/pkl/df2_nb_02_test.pkl.z')
df_submission = pd.read_csv(path + 'Data/sample_submission.csv')
df3_train.drop(df3_train[df3_train[target]==5].index, inplace=True)

df3_train.shape, df3_test.shape

((3999999, 86), (1000000, 85))

In [21]:
df3_train.head()

,Elevation,Aspect,Slope,x_dist_hydrlgy,y_dist_hydrlgy,x_dist_rdwys,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,x_dist_firepts,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,fe_mean,fe_std,fe_median,fe_var,fe_min,fe_max,fe_skew,fe_quantile_25,fe_quantile_50,fe_quantile_75,fe_dammy_count,fe_Hydro_Fire_1,fe_Hydro_Fire_2,fe_Hydro_Road_1,fe_Hydro_Road_2,fe_Fire_Road_1,fe_Fire_Road_2,fe_dist_manh_hydrlgy,fe_dist_ecldn_hydrlgy,fe_soil_type_sum,fe_Soil_Type12_32,fe_Soil_Type23_22_32_33,fe_hillshade_mean,fe_wilderness_area_sum,fe_EVDtH,fe_EHDtH,fe_binned_elevation,fe_pca_0,fe_pca_1,fe_cluster_0,fe_cluster_1,fe_cluster_2,fe_cluster_3
0,3189,40,8,30,13,3270,206,234,193,4873,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1206.0,1832.0,199.5,3356801.250,8,4873,1.272461,32.5,199.5,2450.0,2,4903,4843,3300,3240,8143,1603,44,33.6875,1,0,0,211.000,1,3176,3184.0,63,2124.442770,2935.842506,0,0,1,0
1,3026,182,5,280,29,3270,233,240,106,5423,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,1279.0,1918.0,236.5,3680597.500,5,5423,1.458008,125.0,236.5,2340.0,2,5703,5143,3550,2990,8693,2153,310,118.0625,1,0,0,193.000,1,2997,2970.0,60,2224.754681,3472.999721,0,0,1,0
2,3106,13,7,351,37,2914,208,234,137,5269,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1228.0,1858.0,221.0,3452505.750,7,5269,1.474609,62.0,221.0,2274.0,2,5620,4918,3265,2563,8183,2355,389,81.6250,1,0,0,193.000,1,3069,3036.0,62,1849.005268,3392.884040,0,0,1,0
3,3022,276,13,192,16,3034,207,238,156,2866,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,1002.0,1364.0,222.5,1860983.375,13,3034,1.023438,165.0,222.5,2218.0,2,3058,2674,3226,2842,5900,168,209,169.6250,1,0,0,200.375,1,3006,2984.0,60,1494.519538,1011.880307,0,0,0,1
4,2906,186,13,266,22,2916,231,231,154,2642,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,956.5,1291.0,231.0,1667972.250,13,2916,1.031250,162.0,231.0,2048.0,2,2908,2376,3182,2650,5558,274,289,76.5000,1,0,0,205.375,1,2884,2852.0,58,1330.183635,813.849468,0,0,0,1


In [22]:
df3_train = reduce_memory_usage(df3_train)
df3_test  = reduce_memory_usage(df3_test)

Mem. usage decreased to 488.28 Mb (8.6% reduction)
Mem. usage decreased to 113.49 Mb (9.2% reduction)


# <div class="alert alert-success"> 1.  TUNNING </div>

## 1.0. Split Train/Test

In [23]:
X      = df3_train.drop([target], axis=1)    
y      = df3_train[target].copy()
X_test = df3_test

X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size    = 0.2,
                                                      shuffle      = True, 
                                                      stratify     = y,
                                                      random_state = 12359)

del df3_train , df3_test

free_gpu_cache() 

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape 

((3199999, 85), (3199999,), (800000, 85), (800000,))

## 1.1. Classe Tunning 

In [24]:
class TunningModels(nn.Module):

    from sklearn.preprocessing  import StandardScaler
    from sklearn.linear_model import RidgeClassifier
    
    def __init__(self, name_model_, X_trn_, y_trn_, X_ts_, feature_=None, 
                 seed_=12359, scaler_=StandardScaler(), feature_bin_=None, 
                 target_='target', path_='', level_='1'):
        
        super(TunningModels,self).__init__() 

        self.name_clf    = name_model
        self.X_trn       = X_trn_
        self.y_trn       = y_trn_
        self.X_ts        = X_ts_         
        self.feature     = feature_
        self.seed        = seed_
        self.scaler      = scaler_
        self.feature_bin = feature_bin_ 
        self.target      = target_
        self.path        = path_
        self.level       = level_

    def recover_prediction_first_level():
        
        preds_train1 = glob.glob("model/train/*.pkl.z")
        preds_test   = glob.glob("model/test/*.pkl.z")
        preds_val1   = glob.glob("model/valid/*.pkl.z")

        df_train1     = []
        scores_traint = dict()

        for p_name in preds_train1:    
            p    = jb.load(p_name)
            p_df = pd.DataFrame(p, columns=[p_name.replace('model/train\\', '')])    
            df_train1.append(p_df)    
            scores_traint[p_name] = f1_score(y_train1, (p_df>.5))

        df_val1     = [] 
        scores_val1 = dict()
        for p_name in preds_val1:    
            p    = jb.load(p_name)
            p_df = pd.DataFrame(p, columns=[p_name.replace('model/valid\\', '')])    
            df_val1.append(p_df)    
            scores_val1[p_name] = f1_score(y_val1, (p_df>.5))

        df_test     = [] 
        scores_test = dict()
        for p_name in preds_test:    
            p         = jb.load(p_name)
            p_df_test = pd.DataFrame(p, columns=[p_name.replace('model/test\\', '')])    
            df_test.append(p_df_test)

        df_train1 = pd.concat(df_train1, axis=1)
        df_val1   = pd.concat(df_val1, axis=1)
        df_test   = pd.concat(df_test, axis=1)

        return df_train1, df_val1, df_test.shape
        
    def delete_files(namefile):

        path = ['model/train', 'model/test', 'model/valid', 'model/params', 'model/score',
                'model/test_f', 'model/cv_model', 'model/preds', 'model/optuna', 
                'model/preds/train', 'model/preds/test', 'model/preds/test/n1', 
                'model/preds/test/n2', 'model/preds/test/n3', 'model/preds/train/n1', 
                'model/preds/train/n2', 'model/preds/train/n3','model/preds/param', 
                'Data/submission/tunning', 'Data/submission', 'model/mdl'
                
               ]

        for path_ in path:
            for raiz, diretorios, arquivos in os.walk(path_):
                for arquivo in arquivos:
                    if arquivo.startswith(namefile):
                        os.remove(os.path.join(raiz, arquivo))
 
    def logging_callback(study, frozen_trail):
        prev_best = study.user_attrs.get('prev_best', None)
        if prev_best != study.best_value:
            study.set_user_attr('prev_best', study.best_value)
            print(f"Trail {frozen_trail.number} finished with best value {frozen_trail.value}")

    def df_return_preds_tunning(model_name=None, level=1): 
    
        if level==1: 
            level_ = 'n1'
        else: 
            if level==2:
                level_ = 'n2'
            else: 
                level_ = 'n3'
        
        paths = ['model/preds/test/n1', 'model/preds/train/' + level_ ]    

        if model_name==None: 
            model_name=''
            
        for i, path in enumerate(paths): 

            name_file_pkl     = glob.glob(path + '/'+ model_name + '*.pkl.z')
            dic_preds_mdl_pkl = dict()

            for p_name in name_file_pkl:    
                y_model_pkl_name_col  = p_name.replace(path + '\\', '').replace('.pkl.z','') 
                y_model_pkl           = jb.load(p_name)   

                dic_preds_mdl_pkl[y_model_pkl_name_col] = y_model_pkl

            if i==0: 
                X_test_pred_nivel_1 = pd.DataFrame(dic_preds_mdl_pkl)
            else: 
                X_train_pred_nivel_1 = pd.DataFrame(dic_preds_mdl_pkl)

        X_train_pred_nivel_1['claim'] = y
        
        return X_train_pred_nivel_1, X_test_pred_nivel_1
    
    def feature_select(mdl, feature=[], best_score=0):
    
        best_feature = ''

        for col in df_train1.columns:

            if col not in feature:
                Xtr  = df_train1[feature+[col]].copy()
                Xval = df_val1[feature+[col]].copy()                

                mdl.fit(Xtr, y_train1)

                p = mdl.predict(Xval)
                c = f1_score(y_val1, p)

                if c > best_score:
                    best_score = c
                    best_feature = col 

        return best_score, best_feature

    def permutation_test(mdl, feature_selected):

        dist = []

        for seed in range(100):

            Xtr  = df_train1[feature_selected].copy()
            Xval = df_val1[feature_selected].copy()

            np.random.seed(seed)

            Xtr['random']  = np.random.permutation(Xtr.iloc[:, -1].values)
            Xval['random'] = np.random.permutation(Xval.iloc[:, -1].values)

            mdl.fit(Xtr, y_train1)

            p = mdl.predict(Xval)
            c = f1_score(y_val1, p)

            dist.append(c)

        dist = np.array(dist)

        return dist.max()

    def feature_selected_model(model = RidgeClassifier(alpha=1.) ):
   
        score_feature, best_feature =  TunningModels.feature_select(model)
        print('Score: {:2.4f} => Feature: {}'. format(score_feature*100 , best_feature))

        feature_selected = []
        feature_selected.append(best_feature)

        loop = True

        while loop:

            best_score = TunningModels.permutation_test(model, feature_selected) 
            best_score = best_score + 1e-4

            score_feature, best_feature = TunningModels.feature_select(model, feature=feature_selected, best_score=best_score)
            

            if score_feature <= best_score:  
                print('Fim')
                loop= False
            else: 
                feature_selected.append(best_feature)
                print('Score: {:2.4f} => Feature: {}'. format(score_feature*100 , best_feature))

        return feature_selected
    
    def model_of_diversity_feature_group(model, name_model, X_, y_, X_ts_, feature_imp_num=5, seed_=12359):

        TunningModels.delete_files(name_model)

        model.fit(X_, y_)

        df               = pd.DataFrame()
        df["feature"]    = X_.columns.to_list()
        df["importance"] = model.feature_importances_
        
        df.sort_values("importance", axis=0, ascending=False, inplace=True)

        feature_import = df[:feature_imp_num]['feature'].to_list()
        
        for feature_imp in  feature_import:

            score_                =  0.09
            feature_best          = []
            feature               = X_ts_.columns            
            feature               = [s for s in feature if s not in feature_import]
            feature_number        = len(feature)
            feature_select_number = np.round(np.sqrt(len(feature)))
            feature_number_sample = int(np.round((feature_number/feature_select_number)))
            feature_sample        = []

            print('='*60)
            print(' Divercidade de Grupos de Features => ({})'.format(feature_imp))
            print('='*60)

            for i in  range(0,5):

                feature            = [s for s in feature if s not in feature_sample]
                feature_sample     = pd.Series(feature).sample(feature_number_sample).to_list() 
                name_model_xgb_div = name_model + 'group_fe_' + str(i)   

                feature_sample.append(feature_imp)

                score = TunningModels.cross_valid(model       = model, 
                                                  model_name_ = name_model_xgb_div, 
                                                  X_          = X_, 
                                                  y_          = y_, 
                                                  X_test_     = X_ts_, 
                                                  type_model  = 2, 
                                                  feature     = feature_sample,
                                                  seed        = seed_, 
                                                  tunning     = 1, 
                                                  print_result= False, 
                                                  n_splits    = 3
                                                  )
                if score >.59:
                    create = '*'
                else: 
                    create = ' '

                if score > score_:
                    # score_ = np.abs(score)
                    feature_best.append(feature)
                    print('Score: {:2.5f} =>{} Gr.Feature: {} {}'.format(score, create, i, feature_sample))

                gc.collect()

            print('')
            
        print('')
        print('FIM')
        print('')
        
    def model_of_diversity_feature_one_(model, name_model, seed_=12359):



        score_       =  0.09
        feature_best = []

        print('')
        print('Feature apenas uma')
        print('-'*20)

        TunningModels.delete_files(name_model)

        for feature in X_train.columns:

            name_model_xgb_div = name_model + feature 

            score = TunningModels.cross_valid(model       = model, 
                                              model_name_ = name_model_xgb_div, 
                                              X_          = X, 
                                              y_          = y, 
                                              X_test_     = X_test_sc_qt, 
                                              type_model  = 2, 
                                              feature     = feature,
                                              seed        = seed_, 
                                              tunning     = 1, 
                                              print_result= False, 
                                              n_splits    = 2
                                              )
            if score >.59:
                create = '*'
            else: 
                create = ' '

            if score > score_:
                score_ = np.abs(score)
                feature_best.append(feature)
                print('F1-score: {:2.5f} => {} feature: {}'.format(score, create, feature ))        

        print('')
        print('Feature dupla')
        print('-'*20)

        for feature in feature_best:

            for feature_ in feature_best:
                if feature != feature_:            
                    name_model_xgb_div = name_model + feature + '_' + feature_     

                    score = TunningModels.cross_valid(model       = model, 
                                                      model_name_ = name_model_xgb_div, 
                                                      X_          = X, 
                                                      y_          = y, 
                                                      X_test_     = X_test_sc_qt, 
                                                      type_model  = 2, 
                                                      feature     = [feature, feature_],
                                                      seed        = seed_, 
                                                      tunning     = 1, 
                                                      print_result= False, 
                                                      n_splits    = 2
                                                      )

                    if score >.59:
                        create = '*'
                    else: 
                        create = ' '

                    print('F1-score: {:.4f} => {} feature: {} | {}'.format(score*100, create,  feature, feature_ )) 

        print('')
        print('FIM')
        print('')
    


        from dateutil.relativedelta import relativedelta
        t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
        return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)
        
    def save_data_model(model_, model_name_, path_, y_pred_train_prob_, y_pred_test_prob_,
                        y_pred_test_, score_, seed_, level_='1', target_='target'):
        
        level_ = 'n'+ level_ + '/'

        if score_>.6:          

            path_name_param = path_ + 'model/preds/param/' + model_name_.format(score_, seed_)
            path_name_train = path_ + 'model/preds/train/' + level_ + model_name_.format(score_, seed_)
            path_name_test  = path_ + 'model/preds/test/'  + level_ + model_name_.format(score_, seed_)    
            path_name_model = path_ + 'model/mdl/'         + model_name_.format(score_, seed_)    

            jb.dump(y_pred_train_prob_, path_name_train)
            jb.dump(y_pred_test_prob_, path_name_test)
            jb.dump(model_, path_name_model)
            jb.dump(pd.DataFrame([model_.get_params()]), path_name_param)   

            if score_>.7:
                # Gerar o arquivo de submissão 
                df_submission[target_] = y_pred_test_
                name_file_sub =  path_ + 'Data/submission/tunning/' + model_name_.format(score_, seed_) + '.csv'
                df_submission.to_csv(name_file_sub, index = False)
                
    def diff(t_a, t_b):
        from dateutil.relativedelta import relativedelta
        t_diff = relativedelta(t_b, t_a)  # later/end time comes first!
        return '{h}h {m}m {s}s'.format(h=t_diff.hours, m=t_diff.minutes, s=t_diff.seconds)
        
    def feature_scaler(df_, scaler_=None, feature_bin_=None):
    
        if scaler_!=None: 
            
            #if feature_bin_!=None:
            #    disc = KBinsDiscretizer(n_bins=50, encode='ordinal', strategy='uniform')
            #    df_[feature_bin_] = disc.fit_transform(df_[feature_bin_])

            df_ = pd.DataFrame(scaler_.fit_transform(df_), columns=df_.columns)
    
        return df_

    def cross_valid(model_, model_name_, X_train_, y_train_, X_test_, fold_=5, target_='target', 
            path_='', level_='1', save_predict_=True, print_result_=True, seed_=12359, 
            feature_=None, feature_bin=None, scaler_=StandardScaler(), threshold=.5, print_report_=False 
            ):

        if feature_!=None: 
            X_train_ = X_train_[feature_]
            X_test_  = X_test_[feature_]

        #--------------------------------------------------------  
        # Scorpo de variáveis
        #--------------------------------------------------------

        time_pred_start    = datetime.now()
        preds_valid_f      = {}
        preds_test         = []
        total_auc          = []
        f_scores           = []
        auc_mean           = []
        f1_mean            = []
        lloss_mean         = []
        preds_test         = 0  
        pred_test_prob     = 0
        df_score_history   = pd.DataFrame()
        df_train_pred_fold = pd.DataFrame()
        df_pred_fold       = pd.DataFrame()
        random             = str(np.random.rand(1)[0]).replace('.','')
        model_name_        = model_name_ + '_score_{:2.5f}_{}_' + random + '.pkl.z'
        clf_name           = model_.__class__.__name__
        pri_result         = 92
        learning_rate      = model_.learning_rate         
        le                 = LabelEncoder()
        y_train_           = pd.DataFrame(le.fit_transform(y_train_), columns=[target_])
                                                   
        #--------------------------------------------------------  
        # Início do process de varilidação
        #--------------------------------------------------------
        have_observation=''

        #if dropout_>0: 
        #    is_dropout='*'

        if print_result_:
            num_parallel_tree = 1 #model_.get_params()['num_parallel_tree']
            learning_rate     = model_.learning_rate
            n_estimators      = model_.n_estimators * num_parallel_tree  
            max_depth         = model_.max_depth 
            msg               = 'Training model: {} - seed {} - n_estimators: {} - learning_rate: {} {:2.5f}'

            print('='*pri_result)            
            print(msg.format(clf_name, seed_, n_estimators, max_depth, learning_rate))
            print('='*pri_result)

        kf = StratifiedKFold(n_splits=fold_, random_state=42, shuffle=True)

        for fold,(idx_train, idx_val) in enumerate(kf.split(X_train_, y_train_, groups=y_train_)):

            time_fold_start = datetime.now()

            #--------------------------------------------------------  
            # Seleção dos dados
            #--------------------------------------------------------
            X_trn, X_val = X_train_.iloc[idx_train], X_train_.iloc[idx_val]
            y_trn, y_val = y_train_.iloc[idx_train], y_train_.iloc[idx_val]
            index_valid  = idx_train

            #--------------------------------------------------------  
            # Processamento
            #--------------------------------------------------------        
            X_trn = TunningModels.feature_scaler(X_trn, scaler_, feature_bin) 
            X_val = TunningModels.feature_scaler(X_val, scaler_, feature_bin) 

            #--------------------------------------------------------  
            # Modelo
            #--------------------------------------------------------
            model = model_.fit(X_trn, y_trn,
                               eval_set              = [(X_trn, y_trn), (X_val, y_val)],          
                               early_stopping_rounds = int(n_estimators*.1),
                               verbose               = False)

            #--------------------------------------------------------  
            # oof
            #--------------------------------------------------------
            preds_valid_proba = model.predict_proba(X_val, ntree_limit=model_.best_ntree_limit)
            y_pred_valid      = le.inverse_transform(np.argmax(preds_valid_proba, axis=1))
            
            #--------------------------------------------------------  
            # Obtenha os valores médios de cada fold para a previsão
            #--------------------------------------------------------  
            y_pred_test_prob = model.predict_proba(X_test_, ntree_limit=model_.best_ntree_limit)
            pred_test_prob  += np.max(y_pred_test_prob, axis=1) / fold_
            preds_test      += le.inverse_transform(np.argmax(y_pred_test_prob, axis=1)) / fold_

            #--------------------------------------------------------  
            # Métricas 
            #-------------------------------------------------------- 
            y_val = le.inverse_transform(y_val)
            acc   = metrics.accuracy_score(y_val, y_pred_valid)
            f1    = metrics.f1_score(y_val, y_pred_valid, average='weighted')
            prec  = metrics.precision_score(y_val, y_pred_valid, average='macro')

            #--------------------------------------------------------  
            # Concatenar validação e predição
            #--------------------------------------------------------        
            df_val_pred_fold = pd.DataFrame({'fold'     : fold+1,
                                             'index'    : idx_val, 
                                             'acc'      : acc, 
                                             'f1'       : f1,
                                             'prec'     : prec,                                              
                                             'target'   : y_val, 
                                             'y_pred'   : y_pred_valid, 
                                             'pred_val' : np.max(preds_valid_proba, axis=1)
                                             })

            df_train_pred_fold = pd.concat([df_train_pred_fold, df_val_pred_fold], axis=0)

            col_name        = le.inverse_transform(list(model.classes_))
            df_prob_temp    = pd.DataFrame(preds_valid_proba, columns=col_name)
            y_pred_pbro_max = df_prob_temp.max(axis=1)

            
            df_prob_temp['y_val']     = y_val
            df_prob_temp['y_pred']    = y_pred_valid 
            df_prob_temp['y_proba']   = np.max(preds_valid_proba, axis=1)            
            df_prob_temp['acc']       = acc   
            df_prob_temp['f1']        = f1 
            df_prob_temp['precision'] = prec              
            df_prob_temp['fold']      = fold+1
            df_prob_temp['index']     = idx_val   
            
            df_pred_fold = pd.concat([df_pred_fold, df_prob_temp], axis=0)
            
            del df_prob_temp
            
            #df_prob_temp['scaler']  = str(string_scaler)
            
            #preds_valid_proba = np.max(preds_valid_proba, axis=1)
            
            
            auc_mean.append(acc)   
            f1_mean.append(f1)    
            lloss_mean.append(prec) 

            #--------------------------------------------------------  
            # Print resultado Fold
            #--------------------------------------------------------
            if print_result_:
                msg = 'Fold: {} - ACC: {:2.5f} - F1-score: {:2.5f} - Precision: {:2.5f} - {}'
                time_fold_start_end = TunningModels.diff(time_fold_start, datetime.now())
                print(msg.format(fold+1, acc, f1, prec, time_fold_start_end))

            free_gpu_cache() 
        
        

        del X_trn, y_trn, X_val, y_val 

        df_train_pred_fold.sort_values("index", axis=0, ascending=True, inplace=True)

        #--------------------------------------------------------  
        # Salvar predição em disco
        #--------------------------------------------------------
        X_train_prob      = df_train_pred_fold['pred_val'].to_list()
        score             = np.mean(auc_mean)
        y_pred_test       = np.int32(preds_test)

        if save_predict_:
            TunningModels.save_data_model(model_             = model_, 
                                          model_name_        = model_name_, 
                                          path_              = path_, 
                                          y_pred_train_prob_ = X_train_prob, 
                                          y_pred_test_prob_  = pred_test_prob, 
                                          y_pred_test_       = y_pred_test,
                                          score_             = score, 
                                          seed_              = seed_, 
                                          level_             = level_, 
                                          target_            = target_
                                          )  

        #--------------------------------------------------------  
        # Print média dos Folds
        #--------------------------------------------------------
        time_pred_end = TunningModels.diff(time_pred_start, datetime.now())

        if print_result_:
            msg = '[Mean Fold]  ACC: {:.5f}(Std:{:.5f}) - F1: {:.5f} - Precision: {:.5f}  {}'        
            print('-'*pri_result)            
            print(msg.format(np.mean(auc_mean),np.std(auc_mean) , np.mean(f1_mean), np.mean(lloss_mean), time_pred_end))
            print('='*pri_result)
            print()
            
            if print_report_: 
                y_pred = df_train_pred_fold['y_pred']
                y_vl   = df_train_pred_fold['target']
                print(metrics.classification_report(y_vl, y_pred))

        free_gpu_cache() 

        return model, score, y_pred_test, df_pred_fold        
   
    def xgb(self, trial):
           
        # https://xgboost.readthedocs.io/en/latest/parameter.html
        # https://amangupta16.medium.com/xgboost-hyperparameters-explained-bb6ce580501d        
        params = {'objective'         : trial.suggest_categorical('objective', ['binary:logistic']), 
                  'booster'           : trial.suggest_categorical('booster', ['gbtree']), 
                  'eval_metric'       : trial.suggest_categorical('eval_metric', ['auc']),
                  'n_estimators'      : trial.suggest_int('n_estimators', 500, 800),
                  'max_depth'         : trial.suggest_int('max_depth', 2, 7),            
                  'min_child_weight'  : trial.suggest_int('min_child_weight', 190, 230),  
                  'num_parallel_tree' : trial.suggest_int('num_parallel_tree', 1, 3),
                  'learning_rate'     : trial.suggest_float('learning_rate', .085, 0.121),
                  #'colsample_bylevel' : trial.suggest_float('colsample_bylevel', .90, .95), 
                  #'subsample'         : trial.suggest_float('subsample', .65, .75),                  
                  #'colsample_bynode'  : trial.suggest_float('colsample_bynode', .85, .95),      
                  'sampling_method'   : trial.suggest_categorical('sampling_method', ['gradient_based']), 
                 }
                
        if torch.cuda.is_available():           
            params.update({'predictor'  : trial.suggest_categorical('predictor', ['gpu_predictor']), 
                           'tree_method': trial.suggest_categorical('tree_method', ['gpu_hist']) , 
                           'gpu_id'     : trial.suggest_int('gpu_id', 0,0)})
                
        #pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "validation-auc")
        
        mdl = xgb.XGBClassifier(**params) #, callbacks=[pruning_callback])
        
        _, score, _, _  = TunningModels.cross_valid(model_         = mdl, 
                                                    model_name_    = self.name_clf, 
                                                    X_train_       = self.X_trn, 
                                                    y_train_       = self.y_trn, 
                                                    X_test_        = self.X_ts,            
                                                    target_        = self.target,
                                                    path_          = self.path,
                                                    level_         = self.level,            
                                                    seed_          = self.seed,
                                                    feature_       = self.feature, 
                                                    feature_bin    = self.feature_bin,
                                                    scaler_        = self.scaler                                            
                                                    )
        
        print('param = {}'.format(params))
        print()

        return score

## 1.2. XGB


# https://www.kaggle.com/kaaveland/tps202112-reasonable-xgboost-model

In [25]:
#https://www.kaggle.com/kaggleqrdl/qrdl-covtype-tpsdec21-01-keras-quickstart

In [ ]:
params = {
   # 'num_class': num_class,
    'objective': 'multi:softprob',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'eval_metric': ['merror', 'mlogloss'],
    'learning_rate': .09,
    'max_depth': 0,
    'subsample': .15,
    'sampling_method': 'gradient_based',
    'seed': 64,
    'grow_policy': 'lossguide',
    'max_leaves': 255,
    'lambda': 100,
}

params = {'objective'         : 'multi:softprob', #'multi:softmax',     
          'eval_metric'       : eval_metric,         
          #'sample_weight'     : sample_weights, 
          'num_class'         : len(y.unique()),
          'n_estimators'      : 500, # [500, 800]  
          'max_depth'         : 0,   # [2, 7]
          'max_leaves'        : 255,
          'min_child_weight'  : 190, # [200, 230 ]          
          'num_parallel_tree' : 1,   # [1,3]
          'learning_rate'     : .09,  # [1., .109]
          'colsample_bylevel' : .95, # [.85, 95]
          'subsample'         : .15, # [.65, .75]          
          'colsample_bynode'  : .95, # [.85, .95],  
          'lambda'            : 100,
          'sampling_method'   : 'gradient_based', 
          'grow_policy'       : 'lossguide',
          'random_state'      : SEED}


Estamos configurando isso como um classificador de softprob, o que significa que esperaremos que o booster emita probabilidades em vez de classes. Isso é muito útil para fazer ensembles de votação suave, e também é muito útil para fazer análises de quais samples são difíceis e quais não são. Eu escolho softprob para basicamente todos os classificadores multiclasse com xgboost.

Configurei o xgboost para usar GPU aqui, se não posso usá-lo com GPU, prefiro usar lightgbm. Mas a GPU permite algumas opções interessantes que sempre gosto de experimentar em problemas tabulares, e torna a velocidade de treinamento do booster competitiva com opções como redes neurais e lightgbm.

Quanto ao 'eval_metric', é importante saber que o xgboost usará apenas a última métrica para parada antecipada. Quando fornecido com vários conjuntos de dados de avaliação, ele usará apenas o último conjunto de dados. Nesta competição, descobri que obtenho melhores resultados ao usar logloss para parada antecipada do que erro, mas tentei os dois e você também deve tentar.

A taxa de aprendizado é um parâmetro xgboost com o qual sempre vale a pena brincar. .1 não é escolhido arbitrariamente, mas também não fiz uma grande pesquisa para garantir que seja um grande valor. Em geral, quanto menor for esse número, mais iterações serão necessárias e maiores as chances de que você superejuste. Uma taxa de aprendizado baixa permite que o booster seja muito complexo, então você pode querer usar uma regularização forte se precisar de uma taxa de aprendizado baixa.

max_depth é a profundidade máxima permitida das árvores de decisão que o booster usa. Normalmente, um bom valor aqui é 3-11. Mas, devido a outras opções que estamos usando aqui, podemos permitir com segurança um max_depth infinito (consulte a discussão sobre grow_policy abaixo), outros limites impedirão o xgboost de criar uma árvore infinitamente profunda.

subamostra é definido como .2 aqui, mas provavelmente poderíamos ir tão baixo quanto .1. O significado aqui é a fração das amostras que serão usadas em cada iteração do booster. Ao usar sampling_method = 'gradient_based', você pode definir isso muito baixo, o que é muito útil ao brincar com conjuntos de dados grandes como o que temos aqui. Porém, isso só é possível com tree_method = 'gpu_hist'. É útil definir esse valor baixo, quando você puder - acelera o treinamento significativamente para olhar apenas cerca de 10-20% dos dados em cada iteração.

Eu defini a semente para resultados reproduzíveis e grow_policy = 'lossguide'. O que isso faz é que as árvores cresçam no sentido das folhas, em vez de no sentido da profundidade, por exemplo. não é mais necessário preencher um nível inteiro na árvore de decisão antes de descer ainda mais. Isso significa que devemos definir max_leaves = 255 para algum valor, aqui eu escolhi 255 e remover o limite em max_depth, ou defini-lo realmente alto.

lambda é para regularização l2. Nós o definimos bem alto aqui, já que max_leaves é alto e nossa taxa de aprendizado não é muito alta. Para esta competição, descobri que é importante regularizar os boosters para obter um bom desempenho do LB

Essas configurações que eu escolhi fazem o xgboost agir um pouco como lightgbm, mas como ele roda na GPU, é bastante rápido, mesmo na GTX1660Ti do meu laptop.

Eu trato esses parâmetros como padrões razoáveis, eles não são parâmetros que eu ajustei com uma pesquisa de hiperparâmetro e de forma alguma são ideais. Eles estão muito próximos do que tentarei na primeira vez que encontrar um novo problema tabular. Tenho certeza que com alguns ajustes poderemos obter uma pontuação melhor. Provavelmente temos mais a ganhar brincando com learning_rate, lambda e max_leaves ou "começando de novo" sem grow_policy = 'lossguide' e sampling_method = 'gradient_based'.

Se eu fosse introduzir novas opções, poderia procurar mais regularização aqui, talvez gamma e colsample_bytree. Nesse ponto, pode ser necessário executar mais iterações também.

A seguir, vamos escrever uma função de utilidade que irá misturar os pseudo rótulos em um conjunto de trem e embaralhá-los, já que precisaremos disso para treinar:


In [54]:
int(107686 * .7 + 34215 * 3 + 6327 * 20 + 208 * 500)

408565

In [55]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

sampling_strategy_over = {2: 1244107 , 
                          1: 807457, 
                          3: 107686  + int(107686 * .7), 
                          7: 34215   +  34215 * 3, 
                          6: 6327    +   6327 * 20, 
                          4: 208     +    208 * 500
                         }

sampling_strategy_under = {2: 1244107  - 408565, 
                          1: 807457    - 40000, 
                          3: 107686    + int(107686 * .7), 
                          7: 34215     +  34215 * 3, 
                          6: 6327      +   6327 * 20, 
                          4: 208       +    208 * 500
                         }

oversample  = RandomOverSampler(sampling_strategy=sampling_strategy_over,  random_state=12359)
undersample = RandomUnderSampler(sampling_strategy=sampling_strategy_under,  random_state=12359)

X_over = X.iloc[idx_sample]
y_over = y.iloc[idx_sample]

X_over, y_over = oversample.fit_resample(X_over, y_over)
X_over, y_over = undersample.fit_resample(X_over, y_over)

# 1	1468136	0,38565
# 2	2262087	0,51259
# 3	 195712	0,07817

y_over.value_counts()

2    835542
1    767457
3    183066
7    136860
6    132867
4    104208
Name: Cover_Type, dtype: int64

In [31]:
idx_sample


Int64Index([ 867670,   30845,   34051, 3119854, 2280978, 1848523, 2927911,
            1374148, 3695889, 1747536,
            ...
            3118091, 1213387, 1151278, 1085235,  795769, 3791592, 1781534,
            2815385,  562614, 3882035],
           dtype='int64', length=2200000)

In [39]:
y.iloc[idx_sample].value_counts().to_dict()

{2: 1244107, 1: 807457, 3: 107686, 7: 34215, 6: 6327, 4: 208}

In [ ]:
2    113122
1     73335
3      9969
7      3460
4      2010
6      1515

In [ ]:
df_pred.y_val.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter

counter = Counter(y_train)
print('Before',counter)
# oversampling the train dataset using SMOTE
smt = SMOTE()
#X_train, y_train = smt.fit_resample(X_train, y_train)


X_over = X.iloc[idx_sample]
y_over = y.iloc[idx_sample]

X_train_sm, y_train_sm = smt.fit_resample(X_train, y_train)

counter = Counter(y_train_sm)
print('After',counter)


In [35]:
#idx_sample = X.sample(2200000).index

In [56]:
%%time 

y_unique       = y.unique()
sample_weights = compute_sample_weight(class_weight='balanced', y=y)                 
eval_metric    = ['merror', 'mlogloss']
    
SEED   = 12359
params = {'objective'         : 'multi:softmax', #'multi:softprob', #,     
          'eval_metric'       : eval_metric,         
          #'sample_weight'     : sample_weights, 
          'num_class'         : len(y.unique()),
          'n_estimators'      : 500, # [500, 800]  
          'max_depth'         : 4,   # [2, 7]
          'max_leaves'        : 255,
          'min_child_weight'  : 190, # [200, 230 ]          
          'num_parallel_tree' : 1,   # [1,3]
          'learning_rate'     : .09,  # [1., .109]
          'colsample_bylevel' : .95, # [.85, 95]
          'subsample'         : .15, # [.65, .75]          
          'colsample_bynode'  : .95, # [.85, .95],  
          'lambda'            : 100,
          'sampling_method'   : 'gradient_based', 
          'grow_policy'       : 'lossguide',
          'random_state'      : SEED}

if torch.cuda.is_available():           
    params.update({'predictor'  : 'gpu_predictor', 
                   'tree_method': 'gpu_hist', 
                   'gpu_id'     :  0})    


scaler = MinMaxScaler()

score_best     = 0 
name_model_clf = 'xgb_' 
name_model     = name_model_clf + '001_tunning_under_over_sample' 

TunningModels.delete_files(name_model)

X_test_scaler = TunningModels.feature_scaler(X_test.copy(), scaler)
name_mdl      = name_model #+ str(scaler).lower()[:4]

model, score, _, df_pred  = TunningModels.cross_valid(model_         = xgb.XGBClassifier(**params), 
                                                      model_name_    = name_mdl, 
                                                      X_train_       = X_over, #.iloc[idx_sample], 
                                                      y_train_       = y_over, #.iloc[idx_sample], 
                                                      X_test_        = X_test_scaler, 
                                                      fold_          = 5, 
                                                      target_        = target,
                                                      path_          = path,
                                                      level_         = '1',
                                                      save_predict_  = True, 
                                                      print_result_  = True,
                                                      seed_          = SEED,
                                                      feature_       = None, 
                                                      feature_bin    = None, 
                                                      scaler_        = MinMaxScaler(), 
                                                      threshold      = .5, 
                                                      print_report_  = True
                                                      )

name_model     = name_model_clf + '001_tunning'
name_mdl      = name_model #+ str(scaler).lower()[:4]
model, score, _, df_pred  = TunningModels.cross_valid(model_         = xgb.XGBClassifier(**params), 
                                                      model_name_    = name_mdl, 
                                                      X_train_       = X.iloc[idx_sample], 
                                                      y_train_       = y.iloc[idx_sample], 
                                                      X_test_        = X_test_scaler, 
                                                      fold_          = 5, 
                                                      target_        = target,
                                                      path_          = path,
                                                      level_         = '1',
                                                      save_predict_  = True, 
                                                      print_result_  = True,
                                                      seed_          = SEED,
                                                      feature_       = None, 
                                                      feature_bin    = None, 
                                                      scaler_        = MinMaxScaler(), 
                                                      threshold      = .5, 
                                                      print_report_  = True
                                                      )

Training model: XGBClassifier - seed 12359 - n_estimators: 500 - learning_rate: 4 0.09000
Fold: 1 - ACC: 0.94922 - F1-score: 0.94938 - Precision: 0.93243 - 0h 8m 52s
Fold: 2 - ACC: 0.95088 - F1-score: 0.95082 - Precision: 0.93880 - 0h 9m 23s
Fold: 3 - ACC: 0.95277 - F1-score: 0.95287 - Precision: 0.93704 - 0h 9m 41s
Fold: 4 - ACC: 0.92278 - F1-score: 0.92298 - Precision: 0.91447 - 0h 10m 29s
Fold: 5 - ACC: 0.91840 - F1-score: 0.91913 - Precision: 0.90229 - 0h 8m 56s
--------------------------------------------------------------------------------------------
[Mean Fold]  ACC: 0.93881(Std:0.01498) - F1: 0.93904 - Precision: 0.92501  0h 47m 37s

              precision    recall  f1-score   support

           1       0.96      0.93      0.95    767457
           2       0.95      0.96      0.95    835542
           3       0.85      0.92      0.88    183066
           4       0.98      0.97      0.97    104208
           6       0.91      0.91      0.91    132867
           7       0.90 

In [ ]:
y_val = df_pred['y_val'][df_pred.fold ==1 ]
y_pre = df_pred['y_pred'][df_pred.fold ==1 ]
metrics.accuracy_score(y_val, y_pre), df_pred['acc'][df_pred.fold ==1 ].mean()


In [ ]:
df_pred.sort_values("index", axis=0, ascending=True, inplace=True)
df_pred

In [ ]:
df_pred.y_pred.value_counts()

In [ ]:
2    16920       17186
1    11087       11003
3     1464       1745
7      445       66
6       82
4        2

In [ ]:
y_val = df_pred['y_val'][df_pred.fold ==1 ]
y_pre = df_pred['y_pred'][df_pred.fold ==1 ]
metrics.accuracy_score(y_val, y_pre), df_pred['acc'][df_pred.fold ==1 ].mean()


### 1.2.1. Tunning 
Nesta etapa de modelagem, vamos criar 20 modelos e salvá-los para a nossa `Stacking`.

In [ ]:
%%time

n_trials_  = 50
name_model = name_model_clf + '002_tunning_' 

TunningModels.delete_files(name_model)
    
modelOpt = TunningModels(name_model_     = name_model, 
                         X_trn_          = X, 
                         y_trn_          = y, 
                         X_ts_           = X_test_scaler,                                     
                         feature_        = None,  
                         scaler_         = scaler_best, 
                         seed_           = SEED, 
                         feature_bin_    = None, 
                         target_         = 'target', 
                         path_           = path, 
                         level_          = '1')

pruner = MedianPruner(n_startup_trials = 5,
                      n_warmup_steps   = 0,
                      interval_steps   = 1,
                      n_min_trials     = 5,
                      )

study = optuna.create_study(direction = 'maximize',
                            sampler   = optuna.samplers.TPESampler(seed=SEED),
                            pruner    = optuna.pruners.MedianPruner(n_warmup_steps=10),
                            #pruner    = pruner,
                            study_name= 'xgb_tuning'
                            ) 

study.optimize(modelOpt.xgb, n_trials=n_trials_)

score_seed = study.best_value 
params     = study.best_params 
path_name  = path + 'model/optuna/' + name_model + '_{:2.5f}.pkl.z'.format(score_seed) 
  
scare_best  = score_seed 
params_best = params

print()
print('-'*110)
print('Best score: {:2.5f}'.format(scare_best))
print('Seed      : {}'.format(SEED))
print('Parameters:\n\n{}'.format(params_best))
print()

### 1.2.1. Análise  de Hyperparametros

In [ ]:
plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
plot_slice(study)

In [ ]:
optuna.visualization.plot_slice(study)

### 1.2.3. Modelo Final
Agora que temos os melhores parametros, vamos treinar uma modelo com esse parametros e fazer algumas análises. 

In [ ]:
params_best 

In [ ]:
%%time

name_model = name_model_clf + '003_tun'

model, score, _, _  = TunningModels.cross_valid(model_         = xgb.XGBClassifier(**params_best ), 
                                                model_name_    = name_mdl, 
                                                X_train_       = X_train[feature_], 
                                                y_train_       = y_train, 
                                                X_test_        = X_test_scaler[feature_], 
                                                fold_          = 5, 
                                                target_        = 'target',
                                                path_          = path,
                                                level_         = '1',
                                                save_predict_  = False, 
                                                print_result_  = True,
                                                seed_          = SEED,
                                                feature_       = None, 
                                                #feature_bin    = feature_int, 
                                                scaler_        = scaler_best, 
                                                threshold      =.5                                                      
                                                ) 

### 2.1.3. Feature Importances  

In [ ]:
feature_imp_     = model.feature_importances_

df               = pd.DataFrame()
df["Feature"]    = X[feature_].columns
df["Importance"] = feature_imp_ / feature_imp_.sum()

df.sort_values("Importance", axis=0, ascending=False, inplace=True)

fig, ax = plt.subplots(figsize=(10, 30))
bars    = ax.barh(df["Feature"], 
                  df["Importance"], 
                  height    = 0.4,
                  color     = "mediumorchid", 
                  edgecolor = "black")

ax.set_title("Feature importances \n", fontsize=25, pad=15)
ax.set_ylabel("Feature name", fontsize=15, labelpad=15)
#ax.set_xlabel("Feature importance", fontsize=20, labelpad=15)
ax.set_yticks(df["Feature"])
ax.set_yticklabels(df["Feature"], fontsize=13)
ax.tick_params(axis="x", labelsize=15)
ax.grid(axis="x")

# Adicionando rótulos na parte superior
ax2 = ax.secondary_xaxis('top')
#ax2.set_xlabel("Feature importance", fontsize=20, labelpad=13)
ax2.tick_params(axis="x", labelsize=15)
ax.margins(0.05, 0.01)

# Inverter a direção do eixo y 
plt.gca().invert_yaxis()

In [ ]:
#    def cross_valid_feature_boruta(model, X_, y_, X_ts_, feature_bin, scaler): 
idx_sample_X = X.sample(300000, random_state=SEED).index.to_list()  

model_         = xgb.XGBClassifier(**params_best)
X_train_       = X.iloc[idx_sample_X]
y_train_       = y.iloc[idx_sample_X]
feature_bin_   = None 
scaler_        = scaler_best
fold_          = 4
seed_          = SEED
feature_bin_   = None
feature_select = []


free_gpu_cache()

kf = StratifiedKFold(n_splits=fold_, random_state=seed_, shuffle=True)


for fold,(idx_train, idx_val) in enumerate(kf.split(X_train_, y_train_, groups=y_train_)):

    time_fold_start = datetime.now()

    #--------------------------------------------------------  
    # Seleção dos dados
    #--------------------------------------------------------
    X_trn, X_val = X_train_.iloc[idx_train], X_train_.iloc[idx_val]
    y_trn, y_val = y_train_.iloc[idx_train], y_train_.iloc[idx_val]
    index_valid  = X_val.index.tolist() 

    #--------------------------------------------------------  
    # Processamento
    #--------------------------------------------------------        
    X_trn = TunningModels.feature_scaler(X_trn, scaler_, feature_bin_) 
    X_val = TunningModels.feature_scaler(X_val, scaler_, feature_bin_) 
    
    #--------------------------------------------------------  
    # Modelo de seleção de variáveis
    #--------------------------------------------------------        

    model_feature_selctor = BorutaShap(model= model_, importance_measure = 'shap', classification=True)
    
    model_feature_selctor.fit(X             = X_trn, 
                              y             = y_trn, 
                              n_trials      = 50, 
                              sample        = False,
                              train_or_test = 'test', 
                              normalize     = True,
                              random_state  = seed_)
    
    feature_select.append(sorted(model_feature_selctor.Subset().columns))        

In [ ]:
final_selection = sorted({item for selection in feature_select for item in selection})
print(final_selection)

# Feature Selection by Boruta-SHAP
- https://www.kaggle.com/lucamassaron/feature-selection-by-boruta-shap

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
Observando os dados acima do treinamento, parece que temos um AUC boa, vamos dar uma olhada no gráfico de treinamento do modelo.
    
</div>

In [ ]:
results     = model_002.evals_result() 
ntree_limit = int(model_002.best_ntree_limit / model_002.get_params()['num_parallel_tree'] )

plt.figure(figsize=(7,5))
plt.plot(results["validation_0"]["auc"], label="Treinamento")
plt.plot(results["validation_1"]["auc"], label="Validação")


plt.axvline(ntree_limit, 
            color="gray", 
            label="N. de árvore ideal {}".format(ntree_limit))

plt.xlabel("Número de árvores")
plt.ylabel("AUC")
plt.legend();

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
Estamos treinando o modelo com 707 arvores, mas o ideal é que seja 480, então vamos fazer a predição com o ideal de 480 arvores. 

</div>

In [ ]:
y_hat_002 = model_002.predict_proba(X_valid_sc, ntree_limit=ntree_limit)

#### 1.2.3.1. Analise do Modelo

#### 1.2.3.1.1. Curva Roc

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(y_valid, y_hat_002[:,1])

plot_roc_curve(fpr, tpr, label="XGB")
plt.show()

<div class="alert alert-info" role="alert"> 
    
**`NOTA:`** <br>
 

</div>


In [ ]:
threshold = .5
f1_002  = metrics.f1_score (y_valid, (y_hat_002[:,1] > threshold))
auc_002 = metrics.roc_auc_score(y_valid, y_hat_002[:,1] )
print(metrics.classification_report(y_valid, (y_hat_002 > threshold)[:,1].astype(int) ))
print('')
print('F1-score: {:2.5f}'.format(f1_002))
print('AUC     : {:2.5f}'.format(auc_002))

In [ ]:
plt.figure(figsize=(15,5))
ax = plt.subplot(1,2,1)

metrics.plot_confusion_matrix(model_002, 
                              X_valid_sc, 
                              y_valid, 
                              cmap          = 'inferno', 
                              values_format = 'd', 
                              ax            = ax) #true’, ‘pred’, ‘all’

plt.title('Confusion matrix')

ax= plt.subplot(1,2,2)
metrics.plot_confusion_matrix(model_002, X_valid_sc, y_valid, cmap='inferno', normalize='all', ax=ax) 
plt.title('Confusion matrix');

#### 1.2.3.2. Gerar Submission 

In [ ]:
y_hat_002 = model_002.predict_proba(X_test_scaler, ntree_limit=ntree_limit)

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'target': y_hat_002[:,1]})
submission.to_csv(path + 'Data/submission/xgb_004_0.75420_tunning.csv', index=False)

# score kaggle: 0.85523

### 1.2.4. Divercidade

#### 1.2.4.1. SEED
Nesta etapa vamos utilizar os melhores parametros, que encontramos na tunagem acima, com `seed` diferentes. 

In [ ]:
%%time 

SEED_      = [42, 59, 1000, 1500, 2020, 2021 ]
name_model = 'xgb_001_div_seed'
df_seed    = pd.DataFrame()

#TunningModels.delete_files(name_model)

for i, seed_ in  enumerate (SEED_):     
        
    y_hat = TunningModels.cross_valid(model          = xgb.XGBClassifier(**params_best),  
                                      model_name_    = name_model, 
                                      X_             = X, 
                                      y_             = y, 
                                      X_test_        = X_test_scaler, 
                                      type_model     = 1, 
                                      feature        = None,
                                      seed           = seed_, 
                                      tunning        = 2, # 
                                      scaler         = sc,
                                      is_ntree_limit = feature_int
                                      )
    
    df_seed['seed_' + str(seed_)] = y_hat 
    

#### 1.2.4.2. Gerar submission 

In [ ]:
df_seed.mean(axis=1)

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'target': df_seed.mean(axis=1)})
submission.to_csv(path + 'Data/submission/xgb_005_seed_mean.csv', index=False)

#### 1.2.4.1. Leaning Rate

In [ ]:
%%time 

learning_rate    = [0.01, 0.05, 0.007, ]
name_model       = 'xgb_001_div_learning_rate'
df_learning_rate = pd.DataFrame()
seed             = 1000       

params_xgb_2 = {'max_depth'         : 6,
                'n_estimators'      : 4500,
                'subsample'         : 0.8,
                'colsample_bytree'  : 0.2,
                'learning_rate'     : 0.01,
                'colsample_bylevel' : 0.6000000000000001,
                'min_child_weight'  : 56.41980735551558,
                'reg_lambda'        : 75.56651890088857,
                'reg_alpha'         : 0.11766857055687065,
                'gamma'             : 0.6407823221122686,
                'num_parallel_tree' : 1,
                'booster'           : 'gbtree',
                'eval_metric'       : 'auc',
                'tree_method'       : 'gpu_hist',
                'predictor'         : 'gpu_predictor',
                'use_label_encoder' : False
                }

TunningModels.delete_files(name_model)

for i, learning_rate_ in  enumerate (learning_rate):     
        
    params_xgb_2['learning_rate']=learning_rate_
   
    y_hat = TunningModels.cross_valid(model          = xgb.XGBClassifier(**params_xgb_2),  
                                      model_name_    = name_model, 
                                      X_             = X, 
                                      y_             = y, 
                                      X_test_        = X_test, 
                                      type_model     = 1, 
                                      feature        = None,
                                      seed           = seed, 
                                      tunning        = 2, # 
                                      scaler         = None,
                                      is_ntree_limit = False
                                      )
    
    df_learning_rate['learning_rate_' + str(learning_rate_)] = y_hat 

In [ ]:
df_learning_rate.head() 

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'target': df_learning_rate.mean(axis=1)})
submission.to_csv(path + 'Data/submission/xgb_006_learning_mean.csv', index=False)

#### 1.2.4.3. Max Depth

In [ ]:
%%time 

max_depth    = [1, 2, 4, 8]
name_model   = 'xgb_001_div_max_depth'
df_max_depth = pd.DataFrame()
seed             = 1000       

params_xgb_2 = {'max_depth'         : 1,
                'n_estimators'      : 1000,
                'subsample'         : 0.7,
                'colsample_bytree'  : 0.2,
                'learning_rate'     : .05,
                'colsample_bylevel' : 0.6000000000000001,
                'min_child_weight'  : 56.41980735551558,
                'reg_lambda'        : 75.56651890088857,
                'reg_alpha'         : 0.11766857055687065,
                'gamma'             : 0.6407823221122686,
                'num_parallel_tree' : 1,
                'booster'           : 'gbtree',
                'eval_metric'       : 'auc',
                'tree_method'       : 'gpu_hist',
                'predictor'         : 'gpu_predictor',
                'use_label_encoder' : False
                }

TunningModels.delete_files(name_model)

for i, max_depth_  in  enumerate (max_depth):     
        
    params_xgb_2['max_depth']=max_depth_

    y_hat = TunningModels.cross_valid(model          = xgb.XGBClassifier(**params_xgb_2),  
                                      model_name_    = name_model, 
                                      X_             = X, 
                                      y_             = y, 
                                      X_test_        = X_test, 
                                      type_model     = 1, 
                                      feature        = None,
                                      seed           = seed, 
                                      tunning        = 2, # 
                                      scaler         = None,
                                      is_ntree_limit = False
                                      )
    
    df_max_depth['max_depth_' + str(max_depth_)] = y_hat 

In [ ]:
df_max_depth.head() 

In [ ]:
submission = pd.DataFrame({'id': df_submission.id, 'target': df_max_depth.mean(axis=1)})
submission.to_csv(path + 'Data/submission/xgb_007_max_depth_mean.csv', index=False)

#### 1.2.4.4. learning_rate e xgb_model

In [ ]:

SEED_  = [12359]
score_ = 0 

name_model = 'xgb_001_tun'

param_xgb_1 = {'objective'         : 'binary:logistic', 
               'booster'           : 'gbtree', 
               'eval_metric'       : 'auc', 
               'n_estimators'      : 4024, 
               'max_depth'         : 5, 
               'num_parallel_tree' : 2, 
               'min_child_weight'  : 223, 
               'subsample'         : 0.8476534102142796, 
               'colsample_bynode'  : 0.7492808865792617, 
               'learning_rate'     : 0.04252390552994845, 
               'colsample_bytree'  : 0.5165221701938247, 
               'reg_lambda'        : 13, 
               'reg_alpha'         : 10, 
               'gamma'             : 0.5636400620033356, 
               'random_state'      : 12359, 
               'use_label_encoder' : False, 
               'colsample_bylevel' : 0.6079347926363008, 
               'sampling_method'   : 'gradient_based', 
               'predictor'         : 'gpu_predictor', 
               'tree_method'       : 'gpu_hist', 
               'gpu_id'            : 0
               }

In [ ]:
feature_cat = X.select_dtypes(np.int8).columns.to_list()
feature_cat.remove('fe_count_dammy')
print(feature_cat)

In [ ]:
time_s    = datetime.now()
clf_name  = 'XGBClassifier' #model.__class__.__name__
seed      = 1000
    
msg = 'Training model: {}'#' seed {} - n_estimators: {} - max_depth: {} - {:2.5f} - {}'

enc = TargetEncoder(cols=['f22', 'f43', 'f242', 'f243', 'f244', 'f245', 'f246', 'f247', 'f248', 'f249', 
                          'f250', 'f251', 'f252', 'f253', 'f254', 'f255', 'f256', 'f257', 'f258', 'f259',
                          'f260', 'f261', 'f262', 'f263', 'f264', 'f265', 'f266', 'f267', 'f268', 'f269', 
                          'f270', 'f271', 'f272', 'f273', 'f274', 'f275', 'f276', 'f277', 'f278', 'f279', 
                          'f280', 'f281', 'f282', 'f283', 'f284'])


print('='*90)            
#print(msg.format(clf_name, seed, n_estimators, max_depth, learning_rate, num_parallel_tree))
print(msg.format(clf_name))
print('='*90)

free_gpu_cache()

X_, y_     = X, y
best_ntree = []
auc        = []
f1         = []
lloss      = []
level_     = '1'
threshold  = .5
n_splits   = 5
path       = ''

n_repeats    = 1
y_hat_test   = None
print_result = True
X_train_prob = []
auc          = []
lloss        = []
f1           = []
best_ntree   = []
random       = str(np.random.rand(1)[0]).replace('.','')
model_name_  = 'xgb_999_score_{:2.5f}_{}_' + random + '.pkl.z'  

TunningModels.delete_files('xgb_999_score_')

kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)
   
for fold, (train_idx, valid_idx) in enumerate(kf.split(X_, y_, groups=y_)): 

    time_start   = datetime.now()
    n_estimators = int(param_xgb_1['n_estimators'] * .1)
    
    # Obtenha os dados de treino e 
    # validação através de indice
    X_trn, y_trn = X_.iloc[train_idx], y_.iloc[train_idx]
    X_val, y_val = X_.iloc[valid_idx], y_.iloc[valid_idx]
    
    X_trn = enc.fit_transform(X_trn, y_trn)
    X_val = enc.fit_transform(X_val, y_val)

    model    = xgb.XGBClassifier(**param_xgb_1)
    eval_set = [(X_trn,y_trn), (X_val,y_val)]   
    model_1  = model.fit(X_trn, y_trn, 
                              eval_set              = eval_set,
                              early_stopping_rounds = n_estimators , 
                              verbose               = False
                             )
    
    param_xgb_1['learning_rate'] = .05
    model_2 = xgb.XGBClassifier(**param_xgb_1)
    model_2 = model_2.fit(X_trn, y_trn, 
                          eval_set              = eval_set,
                          early_stopping_rounds = n_estimators, 
                          verbose               = False, 
                          xgb_model             = model_1 
                          )
    
    param_xgb_1['learning_rate'] = .007
    model_3 = xgb.XGBClassifier(**param_xgb_1)
    model   = model_3.fit(X_trn, y_trn, 
                          eval_set              = eval_set,
                          early_stopping_rounds = n_estimators, 
                          verbose               = False, 
                          xgb_model             = model_2 
                          )
    
    best_ntree_ = model.best_ntree_limit
    best_ntree.append(best_ntree_)

    # Predição na validação
    y_hat_val_prob = model.predict_proba(X_val, ntree_limit=best_ntree_)[:, 1] #   
    y_hat_val      = (y_hat_val_prob > threshold).astype(int) 

    X_train_prob.extend(y_hat_val_prob)
    
    # Métricas 
    auc_      = metrics.roc_auc_score(y_val, y_hat_val_prob)                    
    f1_score_ = metrics.f1_score(y_val, y_hat_val)        
    log_loss_ = metrics.log_loss(y_val, y_hat_val_prob)

    # Salvar as métricas
    auc.append(auc_)
    f1.append(f1_score_)
    lloss.append(log_loss_)
    
    
    X_test_    = enc.transform(X_test)
    
    # 1 / n_splits das previsões de saída.
    if y_hat_test is None:                    
        y_hat_test   = model.predict_proba(pd.DataFrame(X_test_))[:, 1]        
    else:        
        y_hat_test  += model.predict_proba(pd.DataFrame(X_test_))[:, 1]

    stop = ' '

    if n_estimators > best_ntree_: 
        stop = '*'
    
    elapsed  = TunningModels.diff(time_start, datetime.now())

    if print_result:
        best_ntree_s = str(best_ntree_) + ((5-len(str(best_ntree_))) * ' ')  
        msg = 'Fold {}/{} AUC: {:2.5f} - F1: {:2.5f} - L. loss: {:2.5f} {} {} {}'
        print(msg.format(fold+1, n_splits*n_repeats, auc_, f1_score_, log_loss_, 
                         stop, best_ntree_s, elapsed))

    del X_trn, y_trn, X_val, y_val              
    gc.collect()

del  X_, y_, X_test_ 
gc.collect()

y_hat_test      /= (n_splits*n_repeats)
auc_mean        = np.mean(auc)
auc_std         = np.std(auc)
lloss_mean      = np.mean(lloss)
f1_mean         = np.mean(f1)
f1_std          = np.std(f1)
best_ntree_mean = int(np.mean(best_ntree))
msg             = '[Mean Fold]  AUC: {:.5f}(Std:{:.5f}) - F1: {:.5f} - L. Loss: {:.5f} * {} {}'

score = TunningModels.save_data_model(model, 
                                      model_name_, 
                                      path, 
                                      X_train_prob, 
                                      y_hat_test, 
                                      auc_mean, 
                                      seed, 
                                      level_ = level_
                                     ) 

elapsed  = TunningModels.diff(time_s, datetime.now())

if print_result:
    print('-'*90)
    print(msg.format(auc_mean, auc_std, f1_mean, lloss_mean, best_ntree_mean, elapsed))
    print('='*90)
    print('')

del X_train_prob, y_hat_test,
free_gpu_cache()

#### 1.2.4.3. FEATURE

In [ ]:
%%time

model_rf_div_04 = RandomForestClassifier(**params_best_seed)
TunningModels.model_of_diversity_feature_group(model_rf_div_04, 'rf_003_div_feature_', 
                                               X, y, X_test, seed_=seed_best)  

### 1.2.5. Ensable 

#### 1.2.5.1. Recuparar dataset
Vamos recuperar todas as previsões do LGBM para gerar um ensable. 

In [ ]:
%%time 
df_train_rf, df_test_rf = TunningModels.df_return_preds_tunning('rf')
df_train_rf.shape, df_test_rf.shape

In [ ]:
yy = df_train_rf['claim']
df_train_rf = df_train_rf.filter(regex=r'.8' , axis=1)
df_test_rf  = df_test_rf.filter(regex=r'.8', axis=1)

df_train_rf['claim'] = yy
df_train_rf.shape, df_test_rf.shape

In [ ]:
df_train_rf.head()

#### 1.2.5.2. Descritiva

In [ ]:
df_test_rf.mean(axis=1).describe()

#### 1.2.5.3. Gerar submission 
Vamos gerar uma submission com a media das previssões, para termos uma ideia de como estamos. 

In [ ]:
y_hat_rf_mean = df_test_rf.mean(axis=1)
submission = pd.DataFrame({'id': df_submission.id, 'claim': y_hat_rf_mean })
submission.to_csv(path + 'Data/sumbmission/rf_003_feature_gr.csv', index=False) 
# score kaggle: 0.60423

In [ ]:
y_hat_rf_mean

#### 1.2.5.4. Correlação

In [ ]:
graf_corr(df_train_rf)

Temos muitas previsões autocorrelacionadas, vamos fazer a exclusão de algumas.

In [ ]:
corr_features = correlation(df_train_rf, 0.9)
len(set(corr_features))

In [ ]:
#df_train_lgbm.drop(labels=corr_features, axis=1, inplace=True)

graf_corr(df_train_rf) 

In [ ]:
X_rf      = df_train_rf.drop(['claim'], axis=1)    
y_rf      = df_train_rf["claim"].copy()
X_test_rf = df_test_rf

X_train_rf, X_valid_rf, y_train_rf, y_valid_rf = train_test_split(X_rf, 
                                                                      y_rf, 
                                                                      test_size    = 0.2, 
                                                                      shuffle      = True, 
                                                                     # stratify     = y_rf,
                                                                      random_state = 12359)

print(X_train_rf.shape, y_train_rf.shape, X_valid_rf.shape, y_valid_rf.shape, X_rf.shape, 
      y_rf.shape, X_test_rf.shape) 


In [ ]:
%%time

model_rf_003      = RandomForestClassifier(**params_best_seed)
                                        
y_hat_test_rf_003 = TunningModels.cross_valid(model       = model_rf_003, 
                                               model_name_ = name_model_rf, 
                                               X_          = X_rf, 
                                               y_          = y_rf, 
                                               X_test_     = X_test_rf, 
                                               type_model  = 2, 
                                               feature     = None,
                                               seed        = SEED, 
                                               tunning     = 0
                                               )

model_rf_003 = model_rf_003.fit(X_train_rf, y_train_rf)
y_hat_rf_003 = model_rf_003.predict_proba (X_valid_rf)

In [ ]:

import numpy as np
import pandas as pd
train = pd.read_csv('../input/titanic/train.csv')
train['Fare'].fillna(train[train.Pclass==3]['Fare'].median(),inplace=True)

def Iso_outliers(df):
    iso = IsolationForest( behaviour = 'new', random_state = 1, contamination= 'auto')
    preds = iso.fit_predict(df.values.reshape(-1,1))
    data = pd.DataFrame()
    data['cluster'] = preds
    print(data['cluster'].value_counts().sort_values(ascending=False))
Iso_outliers(train['Fare']) 

In [ ]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest(random_state = 1, contamination= 'auto')
preds = iso.fit_predict(X_train)

In [ ]:
data = pd.DataFrame()
data['IsolationForest'] = preds

In [ ]:
print(data['IsolationForest'].value_counts().sort_values(ascending=False))

In [ ]:
db = DBSCAN(eps=3.0, min_samples=10).fit(X)
labels = db.labels_

In [ ]:
from sklearn.ensemble import IsolationForest
>>> X = [[-1.1], [0.3], [0.5], [100]]
>>> clf = IsolationForest(random_state=0).fit(X)
>>> clf.predict([[0.1], [0], [90]])

In [ ]:
idade   = [10,18,20,36, 31, 56]
salario = [100, 122, 100,1000,70, 100]
x = {'idade': idade, 'salario':salario}

df = pd.DataFrame(x)

df

In [ ]:
clf = IsolationForest(random_state = 1, contamination= 'auto')
pred = clf.fit_predict(df)
df['outliers'] =  pred

In [ ]:
df [df.outliers==1].drop('outliers', axis=1)